In [7]:
import os
import sys
import pathlib
import numpy as np
import tensorflow as tf
from scipy.io.wavfile import read
import wave
from sklearn.model_selection import train_test_split
import librosa
import sounddevice as sd
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import  Conv1D, GRU, Dropout, Input, Dense, Conv2D, Reshape, MaxPooling2D
import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from scipy import linalg
from pydub import AudioSegment


model = tf.keras.Model()
model = tf.keras.models.load_model('../vehicle_model_1D_CRNN/')
longest_sample = 125611

for sample in ['car_sample.wav','tram_sample.wav']:
        audio, fs = librosa.load(os.path.join('../test_model/', sample), sr=16000)
        audio = np.hstack((audio, np.zeros(longest_sample - len(audio))))
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=16000, n_fft=512, hop_length=160, n_mels=48, fmax=8000)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).T
        mel_spec_db_norm = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db) - np.min(mel_spec_db)).T

        prediction = model.predict(np.expand_dims(mel_spec_db_norm.T, axis=0), batch_size=4)
        print(prediction)
        if prediction[:, 1] > 0.50:
                print(f"Car detected")
                print("Confidence:", prediction[:, 1])
        else:
                print(f"Tram detected")
                print("Confidence:", prediction[:, 0])
        sd.play(audio,16000)
        

1/1 [==============================] - 0s 183ms/step
[[3.834071e-04 9.996166e-01]]
Car detected
Confidence: [0.9996166]
1/1 [==============================] - 0s 12ms/step
[[9.999944e-01 5.571220e-06]]
Tram detected
Confidence: [0.9999944]
